Tasks

1

2

3 Within the notebook, discuss the classification of each variable in the dataset based on common variable types and measurement scales in mathematics, statistics, and Python.

4 Select, demonstrate, and explain the most appropriate summary statistics to describe each variable.

5 Select, demonstrate, and explain the most suitable plot(s) for each variable.

Project